In [1]:
from collections import defaultdict, Counter
import os, operator
from progressbar import progressbar as pb
from nltk import word_tokenize
from typing import List, Tuple
from copy import deepcopy

from torch.utils.data import DataLoader
from transformers import BertTokenizer, BertForMaskedLM
import torch
import numpy as np
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score

import gdown #! pip install gdown

In [7]:
! gdown https://drive.google.com/uc?id=1-4fS0lwbS1fZcGmlOn2ro81X68zkQ8ZX

Downloading...
From: https://drive.google.com/uc?id=1-4fS0lwbS1fZcGmlOn2ro81X68zkQ8ZX
To: /Users/alex/Python/multiconer/my_experiments/models/template_free/sbert_large_nlu_ru.zip
100%|██████████████████████████████████████| 1.59G/1.59G [02:24<00:00, 11.0MB/s]


In [11]:
! unzip sbert_large_nlu_ru.zip

Archive:  sbert_large_nlu_ru.zip
   creating: sbert_large_nlu_ru/
  inflating: sbert_large_nlu_ru/.DS_Store  
  inflating: __MACOSX/sbert_large_nlu_ru/._.DS_Store  
  inflating: sbert_large_nlu_ru/config.json  
  inflating: __MACOSX/sbert_large_nlu_ru/._config.json  
  inflating: sbert_large_nlu_ru/pytorch_model.bin  
  inflating: __MACOSX/sbert_large_nlu_ru/._pytorch_model.bin  


In [2]:
data_path = "../../../SemEval2022-Task11_Train-Dev/RU-Russian/"
pretrained = "sberbank-ai/sbert_large_nlu_ru"
SAVE_PATH = "sbert_large_nlu_ru/"
device = 'cpu'

In [3]:
with open(os.path.join(data_path, "ru_dev.conll")) as f:
    dev_file = f.read().splitlines()

In [4]:
def parse_conll(file) -> Tuple[List, List]:
    texts, labels = [], []
    
    for row in file:
        if row.startswith("#"):
            new_texts, new_labels = [], []
            continue

        if row == "":
            texts.append(new_texts)
            labels.append(new_labels)

        else:
            parts = row.split()
            new_texts.append(parts[0])
            new_labels.append(parts[-1])

    return texts, labels

dev_texts, dev_labels = parse_conll(dev_file)

In [5]:
len(dev_texts)

800

In [6]:
label2word_label = { # label words dict
    "GRP": "колхоз",
    "PER": "человек",
    "CW": "сингл",
    "PROD": "dvd",
    "CORP": "mtv",
    "LOC": "париж"
}

word_label2label = {v:k for k, v in label2word_label.items()}

In [7]:
def prepare_target(tokens, labels, label2word_label=label2word_label):
    """
        Replace entities with label words
    """
    new_tokens = []
    for token, label in zip(tokens, labels):
        if label.startswith("B-"):
            prefix, tag = label.split("-")
            new_token = label2word_label[tag]
            new_tokens.append(new_token)
        elif label.startswith("I-"):
            continue
        else:
            new_tokens.append(token)
    
    return new_tokens

In [8]:
dev_data = []
    
for tokens, label_list in zip(dev_texts, dev_labels):
    target = prepare_target(tokens, label_list)
    x = " ".join(tokens)
    y = " ".join(target)
    dev_data.append((x,y))

In [9]:
dev_id_to_entity_map = {}

for i, (text, labels) in enumerate(zip(dev_texts, dev_labels)):
    dev_id_to_entity_map[i] = defaultdict(list)
    for token, label in zip(text, labels):
        if label == "O":
            continue
        prefix, tag = label.split("-")
        if prefix == "B":
            dev_id_to_entity_map[i][tag].append([])
            dev_id_to_entity_map[i][tag][-1].append(label)
        elif prefix == "I":
            dev_id_to_entity_map[i][tag][-1].append(label)

In [10]:
ind = 2
print(dev_texts[ind], dev_labels[ind])

['специальный', 'агент', 'секретной', 'службы', 'сша', 'джеззи', 'фланниган', ',', 'ответственная', 'за', 'нарушение', 'безопасности', ',', 'объединяется', 'с', 'кроссом', ',', 'чтобы', 'найти', 'пропавшую', 'девушку', '.'] ['O', 'O', 'B-GRP', 'I-GRP', 'I-GRP', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [11]:
dev_id_to_entity_map[ind]

defaultdict(list, {'GRP': [['B-GRP', 'I-GRP', 'I-GRP']]})

# Load model

In [12]:
model = BertForMaskedLM.from_pretrained(SAVE_PATH, local_files_only=True)
model.to(device)
model.eval()
print()

In [13]:
tokenizer = BertTokenizer.from_pretrained(pretrained, local_files_only=False)
max_len = 64

In [14]:
def get_pred(text, model=model):
    tokenized = torch.LongTensor([
        tokenizer.encode(text, max_length=max_len, padding="max_length", truncation=True)
    ]).to(device)
    out = model(tokenized).logits
    out = torch.argmax(out, dim=2)
    return tokenizer.decode(out[0], skip_special_tokens=True) 

In [15]:
dev_x = [item[0] for item in dev_data]
dev_y_true = [item[1] for item in dev_data]
dev_y_pred = [get_pred(item) for item in pb(dev_x)]

100% (800 of 800) |######################| Elapsed Time: 0:12:12 Time:  0:12:12


In [16]:
ind = 7
dev_y_true[ind], dev_y_pred[ind]

('выпущен 15 февраля 2019 года лейблом mtv .',
 'выпущен 15 февраля 2019 года леиблом mtv.')

# Examples

### Good

In [17]:
good_idx = [7, 107, 223, 654]

In [18]:
for idx in good_idx:
    print(f"True: {dev_y_true[idx]}")
    print(f"Pred: {dev_y_pred[idx]}\n")

True: выпущен 15 февраля 2019 года лейблом mtv .
Pred: выпущен 15 февраля 2019 года леиблом mtv.

True: в 1968 — 1969 годах играл за команду класса « б » колхоз .
Pred: в 1968 — 1969 годах играл за команду класса « б » колхоз

True: сан педро де атакама ( ) — посёлок в париж .
Pred: сан педро де ударама ( ) — поселок в париж.

True: париж — река во владимирской области россии , приток войнинги .
Pred: париж — река во владимирскои области россии, приток воининги.



### Bad

In [19]:
bad_idx = [22, 780, 2, 674]

In [20]:
for idx in bad_idx:
    print(f"True: {dev_y_true[idx]}")
    print(f"Pred: {dev_y_pred[idx]}\n")

True: по собственному признанию есть три сми , которые тиган ценит с профессиональной точки зрения : mtv , колхоз , and колхоз .
Pred: по собственному признанию есть три сми, которые тиган ценил с профессиональнои точки зрения : mtv, сингл

True: dvd сверху красновато коричневая , по бокам более светлая .
Pred: dvdd красновато коричневая по по по более светлая.

True: специальный агент колхоз джеззи фланниган , ответственная за нарушение безопасности , объединяется с кроссом , чтобы найти пропавшую девушку .
Pred: специальныи сингл секретно за за за за, за за за засяся,,,,м,,вшую..

True: « андрей полисадов » — поэма человек .
Pred: « сингл » — поэма анд андрея вознес..



# Decode predictions and calc metrics

In [21]:
def decode_pred(y_true, y_pred):
    pred_labels = []

    y_pred = y_pred.replace("ё", "е").replace("й", "и")
    y_pred = word_tokenize(y_pred)
    
    y_true = y_true.replace("ё", "е").replace("й", "и")
    y_true = word_tokenize(y_true)
    
    true_labels = [word_label2label.get(token, "O") for token in y_true]
    pred_labels = [word_label2label.get(token, "O") for token in y_pred]
    
    true_len = len(true_labels)
    pred_len = len(pred_labels)
    
    if true_len > pred_len:
        pred_labels.extend(["O"] * (true_len - pred_len))
    elif pred_len > true_len:
        true_labels.extend(["O"] * (pred_len - true_len))
            
    assert len(true_labels) == len(pred_labels)
    return true_labels, pred_labels

In [22]:
dev_id_to_entity_map_cp = deepcopy(dev_id_to_entity_map)

final_dev_pred = []

for i, (t, p, labels) in enumerate(zip(dev_y_true, dev_y_pred, dev_labels)):
    t, p  = decode_pred(t, p)
    fin = []

    for label in p:
        if label == "O":
            fin.append("O")
        else:
            if label in dev_id_to_entity_map_cp[i]:
                fin.extend(dev_id_to_entity_map_cp[i][label][0])
                if len(dev_id_to_entity_map_cp[i][label]) > 1:
                    dev_id_to_entity_map_cp[i][label] = dev_id_to_entity_map_cp[i][label][1:]
            else:
                fin.append("B-" + label)
    
    true_len = len(labels)
    pred_len = len(fin)
    
    if true_len > pred_len:
        fin.extend(["O"] * (true_len - pred_len))
    elif pred_len > true_len:
        fin = fin[:true_len]
    assert len(fin) == len(labels)
    final_dev_pred.append(fin)

In [23]:
flat_true = [item for sublist in dev_labels for item in sublist]
flat_pred = [item for sublist in final_dev_pred for item in sublist]

flat_true = [item.replace("I-", "").replace("B-", "") for item in flat_true]
flat_pred = [item.replace("I-", "").replace("B-", "") for item in flat_pred]
assert len(flat_true) == len(flat_pred)

### Micro Avg. on all classes

In [24]:
result = {}
avg = "micro"

result["ALL"] = {
    "precision": precision_score(flat_true, flat_pred, average=avg),
    "recall": recall_score(flat_true, flat_pred, average=avg),
    "f1": f1_score(flat_true, flat_pred, average=avg)
}

### By class

In [25]:
for label in label2word_label.keys():
    precision = precision_score(flat_true, flat_pred, labels=[label], average=avg)
    recall = recall_score(flat_true, flat_pred, labels=[label], average=avg)
    f1 = f1_score(flat_true, flat_pred, labels=[label], average=avg)
    
    result[label] = {
        "precision": precision,
        "recall": recall,
        "f1": f1
    }

In [26]:
result

{'ALL': {'precision': 0.9138471177944862,
  'recall': 0.9138471177944862,
  'f1': 0.9138471177944862},
 'GRP': {'precision': 0.8305084745762712,
  'recall': 0.5017064846416383,
  'f1': 0.6255319148936169},
 'PER': {'precision': 0.7034883720930233,
  'recall': 0.6269430051813472,
  'f1': 0.663013698630137},
 'CW': {'precision': 0.6398809523809523,
  'recall': 0.617816091954023,
  'f1': 0.6286549707602339},
 'PROD': {'precision': 0.776,
  'recall': 0.46859903381642515,
  'f1': 0.5843373493975903},
 'CORP': {'precision': 0.825,
  'recall': 0.5913978494623656,
  'f1': 0.6889352818371607},
 'LOC': {'precision': 0.6988636363636364,
  'recall': 0.48616600790513836,
  'f1': 0.5734265734265734}}